In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.callbacks import ModelCheckpoint



In [6]:
# Essential and common packages
import os
import glob

# Read and visualize the raster data
import rasterio
from rasterio.plot import show

# Plots and bars
import matplotlib.pyplot as plt
import seaborn as sns

# Computation library
import numpy as np

# Tensorflow for building the resnet50 model
import tensorflow.python.keras as k
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.initializers import random_uniform, glorot_uniform
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


# Sklearn for confusion matrix
from sklearn.utils.class_weight import compute_class_weight

import itertools
# For visualization of plots without plt.show()
%matplotlib inline

dataset_url =r'C:\Users\abudh\Desktop\CropWatch\EuroSAT\2750'
batch_size = 50
img_height = 64
img_width = 64
validation_split=0.2
rescale=1.0/255
no_epochs = 20


# 3. Data Augmentation and Loading
datagen = ImageDataGenerator(
    rescale=rescale,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=0.2,
    fill_mode='nearest',
    validation_split=validation_split
)

dataset = tf.keras.preprocessing.image_dataset_from_directory(dataset_url, image_size=(img_height, img_width), batch_size=batch_size)

train_dataset = datagen.flow_from_directory(batch_size=batch_size,
                                           directory=dataset_url,
                                           shuffle=True,
                                           target_size=(img_height, img_width),
                                           subset="training",
                                           class_mode='categorical')


test_dataset = datagen.flow_from_directory(batch_size=batch_size,
                                           directory=dataset_url,
                                           shuffle=True,
                                           target_size=(img_height, img_width),
                                           subset="validation",
                                           class_mode='categorical')

from sklearn.utils.class_weight import compute_class_weight

class_indices = train_dataset.class_indices
class_labels = list(class_indices.keys())
class_indices = list(class_indices.values())

class_counts_dict = {class_name: len(glob.glob(os.path.join(dataset_url, class_name, '*'))) for class_name in class_labels}
class_counts = np.array([class_counts_dict[class_name] for class_name in class_labels])

weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array(class_indices),
    y=np.array([train_dataset.classes[i] for i in range(len(train_dataset.classes))])
)

class_weight_dict = dict(zip(class_labels, weights))
print("Class weights:")
for label, weight in class_weight_dict.items():
    print(f"Class '{label}': {weight:.4f}")


class_names = dataset.class_names
no_class= (len(class_names))





def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value.
    X_shortcut = X
    cache = []
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1, 1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = X = Activation('relu')(X, training = training)

    return X


def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1, 1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1, 1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s, s), padding = 'valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training = training)

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

def ResNet50(input_shape = (64, 64, 3), classes = len(class_names)):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
   # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size = (2, 2), name = 'avg_pool')(X)
    

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model





from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=15,
    verbose=1,
    min_lr=1e-6
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,
    verbose=1,
    restore_best_weights=True
)




# Define the path where the weights will be saved
model_path = r'C:\Users\abudh\Desktop\CropWatch'

# Define the checkpoint callback to save weights every 5 epochs with a dynamic name
dynamic_checkpoint_callback = ModelCheckpoint(
    filepath=model_path + r'\Training_trained_model_epoch{epoch:02d}.weights.h5',  # Use .weights.h5 extension
    save_freq=5,  # Save at the end of every epoch
    save_weights_only=True,  # Save only weights
    verbose=1  # Verbose mode to show saving information
)

# Define the checkpoint callback to save only the best weights based on validation loss
best_model_checkpoint_callback = ModelCheckpoint(
    filepath=model_path + r'\best_trained_weights.weights.h5',  # Use .weights.h5 extension
    monitor='val_loss',  # Monitor validation loss
    mode='min',  # Minimize the loss
    save_best_only=True,  # Save only the best weights
    save_weights_only=True,  # Save only weights
    verbose=1  # Verbose mode to show saving information
)


model = ResNet50(input_shape=(img_height, img_width, 3), classes=len(class_labels))

model.load_weights(r'C:\Users\abudh\Desktop\CropWatch\best_weights.weights.h5')

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

#history = model.fit(train_dataset, validation_data=test_dataset, epochs=no_epochs)
history = model.fit(
    train_dataset,
    epochs=no_epochs,
    validation_data=test_dataset,
    class_weight=class_weight_dict,
    callbacks=[reduce_lr, early_stopping, dynamic_checkpoint_callback, best_model_checkpoint_callback]
)











Found 27000 files belonging to 10 classes.
Found 21600 images belonging to 10 classes.
Found 5400 images belonging to 10 classes.
Class weights:
Class 'AnnualCrop': 0.9000
Class 'Forest': 0.9000
Class 'HerbaceousVegetation': 0.9000
Class 'Highway': 1.0800
Class 'Industrial': 1.0800
Class 'Pasture': 1.3500
Class 'PermanentCrop': 1.0800
Class 'Residential': 0.9000
Class 'River': 1.0800
Class 'SeaLake': 0.9000
Epoch 1/20


C:\Users\abudh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  4/432 ━━━━━━━━━━━━━━━━━━━━ 9:08 1s/step - accuracy: 0.9558 - loss: 0.1093  
Epoch 1: saving model to C:\Users\abudh\Desktop\CropWatch\Training_trained_model_epoch01.weights.h5
  9/432 ━━━━━━━━━━━━━━━━━━━━ 9:31 1s/step - accuracy: 0.9401 - loss: 0.1537
Epoch 1: saving model to C:\Users\abudh\Desktop\CropWatch\Training_trained_model_epoch01.weights.h5
 14/432 ━━━━━━━━━━━━━━━━━━━━ 9:33 1s/step - accuracy: 0.9352 - loss: 0.1692
Epoch 1: saving model to C:\Users\abudh\Desktop\CropWatch\Training_trained_model_epoch01.weights.h5
 19/432 ━━━━━━━━━━━━━━━━━━━━ 9:30 1s/step - accuracy: 0.9341 - loss: 0.1734
Epoch 1: saving model to C:\Users\abudh\Desktop\CropWatch\Training_trained_model_epoch01.weights.h5
 24/432 ━━━━━━━━━━━━━━━━━━━━ 9:26 1s/step - accuracy: 0.9335 - loss: 0.1766
Epoch 1: saving model to C:\Users\abudh\Desktop\CropWatch\Training_trained_model_epoch01.weights.h5
 29/432 ━━━━━━━━━━━━━━━━━━━━ 9:19 1s/step - accuracy: 0.9338 - loss: 0.1780
Epoch 1: saving model to C:\Users\abudh\De

OSError: [Errno 28] Can't write data (file write failed: time = Sat Aug 10 07:17:07 2024
, filename = 'C:\Users\abudh\Desktop\CropWatch\Training_trained_model_epoch05.weights.h5', file descriptor = 3, errno = 28, error message = 'No space left on device', buf = 000002702DB7C040, total write size = 2359296, bytes this sub-write = 2359296, bytes actually written = 18446744073709551615, offset = 147676704)

In [ ]:
from sklearn.metrics import confusion_matrix
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16.53, 11.69))
ax1.plot(history.history['accuracy'])
ax1.plot(history.history['val_accuracy'])
ax1.set_xlabel('epoch')
ax1.set_ylabel('accuracy')
ax1.set_title('Accuracy over epoch')
ax1.legend(['Train', 'Test'], loc='upper left')

ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_xlabel('epoch')
ax2.set_ylabel('loss')
ax2.set_title('Loss over epoch')
ax2.legend(['Train', 'Test'], loc="upper right")
plt.show()

y_pred = []  # store predicted labels
y_true = []  # store true labels

# iterate over the dataset
for i, (image_batch, label_batch) in enumerate(test_dataset):   # use dataset.unbatch() with repeat
    # append true labels
    y_true.append(label_batch)
    # compute predictions
    preds = model.predict(image_batch)
    # append predicted labels
    y_pred.append(np.argmax(preds, axis =  1))
    if i==300:
        break

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
correct_labels = np.argmax(correct_labels, axis=1)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)
cm = confusion_matrix(correct_labels, predicted_labels)
cm

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()